<h1>Capstone Project</h1>

<h2>Stock Prediction Using Twitter Sentiment Analysis</h2>

In [2]:
'''
load the dataset DJIA dataset(June 2009 to December 2009)
'''

import os
import numpy as np
import pandas as pd
from IPython.display import display

data = open('HistoricalPrices.csv', 'rb')

df = pd.read_csv(data)
display(df.head(5))

,Date,Open,High,Low,Close
0,12/16/09,10449.81,10509.97,10427.59,10441.12
1,12/15/09,10499.31,10499.31,10426.69,10452.00
2,12/14/09,10471.28,10514.66,10471.28,10501.05
3,12/13/09,10420.38,10491.70,10417.88,10478.89
4,12/12/09,10437.35,10499.35,10435.68,10486.27


In [3]:
df.iloc[3]['Date']

'12/13/09'

In [4]:
'''
get all the tweets
'''

tweets = open('TaggedData.csv', 'rb')
thead = pd.read_csv(tweets, usecols=['Date', 'Name', 'Tweets', 'sentiment'])
display(thead.head(5))

,Date,Name,Tweets,sentiment
0,17 Jun 2009,Rodney Holston,is taking this day to relax and catch up on sl...,1
1,17 Jun 2009,Angie,went to chiro today very sore now but know i w...,0
2,17 Jun 2009,Jaxon Brooks,updated the iphone 2g to os 3 0 feels zippier ...,0
3,17 Jun 2009,K,i am sooo hyped for this weekend going to va f...,1
4,17 Jun 2009,Nicola Atkins,was feeling down earlier but two wonderful fri...,1


In [43]:
import datetime

data_file = open("daily_twitter_sentiment.csv", "r")
dhead = pd.read_csv(data_file, usecols = ['Date', 'pos', 'neg'])
data = []
for index,row in dhead.iterrows():
    rec = {}
    rec['Date'] = datetime.datetime.strftime(datetime.datetime.strptime(row['Date'], "%d %b %Y"), "%Y-%m-%d")
    total = row['pos'] + row['neg']
    rec['pos'] = round(float(row['pos'])/total, 4)
    rec['neg'] = round(float(row['neg'])/total, 4)
    data.append(rec)

In [44]:
data[:10]

[{'Date': '2009-06-18', 'neg': 0.6436, 'pos': 0.3564},
 {'Date': '2009-06-19', 'neg': 0.6369, 'pos': 0.3631},
 {'Date': '2009-06-20', 'neg': 0.6488, 'pos': 0.3512},
 {'Date': '2009-06-21', 'neg': 0.6476, 'pos': 0.3524},
 {'Date': '2009-06-22', 'neg': 0.6476, 'pos': 0.3524},
 {'Date': '2009-06-23', 'neg': 0.6623, 'pos': 0.3377},
 {'Date': '2009-06-24', 'neg': 0.6601, 'pos': 0.3399},
 {'Date': '2009-06-25', 'neg': 0.647, 'pos': 0.353},
 {'Date': '2009-06-26', 'neg': 0.747, 'pos': 0.253},
 {'Date': '2009-06-27', 'neg': 0.6692, 'pos': 0.3308}]

In [45]:
df = pd.DataFrame(data)
df.to_csv("daily_sentiment_index.csv")